## Q1)  이 경우 Type 1 error와 Type 2 error 중 무엇이 더 높을까요?

FP는 실제 답이 False일때 True라고 예측 즉 긍정을 잘못한 것, FN은 실제 답이 True일때 False라고 예측 즉 부정을 잘못한 것.

실제로 참인 귀무가설을 잘못 기각하는 경우가 type 1 error고, 실제로 거짓인 귀무가설을 기각하지 않는 경우가 type 2 error다.

threshold가 낮은 경우 FP가 높고 FN이 낮을 것이다. 따라서 type 2 error가 더 높다.

## Q2-1) Accuracy, Precision and Recall이 무엇인지 TP,FP,FN,TN의 식으로 나타내 주세요.
Precision은 정밀도로, True로 분류된 것 중 실제 True의 비율이다. TP/(TP+FP)

Recall은 재현율로, 실제 True 중 모델이 True로 분류한 것의 비율이다. TP/(TP+FN)

Accuracy는 정확도로, True를 True로 False를 False로 옳게 예측한 경우를 둘다 고려하는 지표이다. (TP+TN)/(TP+TN+FP+FN) 

## Q2-2) Precision and Recall에 관한 실생활 예시 하나를 들어주세요. 
임신 판정에 대해 생각해보자. 이 경우 임신한 사람을 임신하지 않았다고 판단하는 FN이 임신하지 않은 사람을 임신했다고 하는 FP보다 더 위험하다. 따라서 precision보다 recall을 우선시해야 하며, threshold를 낮춰서 FN이 낮도록 하는 것이 합리적일 것 같다. 

## Q3) 코드 따라하고 주석 달기

In [1]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 

In [4]:
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 

# Hyper Parameters  
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001
  
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False) 

In [5]:
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 

In [6]:
model = LogisticRegression(input_size, num_classes) 

In [7]:
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [9]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad() # Reset all gradients to 0
        outputs = model(images) # Make a forward pass
        loss = criterion(outputs, labels) # Calculate the loss
        loss.backward() # Perform backpropagation
        optimizer.step() # Update all weights
        
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch+1, num_epochs, i+1, len(train_dataset) // batch_size, loss.data))

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.1184
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 1.9859
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 1.8862
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.8568
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.7613
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.7514
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.6718
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.6260
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.5068
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5223
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.4675
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4272
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.3553
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.3728
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.3420
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.2926
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2508
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2172
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.1916
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1581
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [10]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % ( 100 * correct / total)) 

..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Accuracy of the model on the 10000 test images:  82 %


# Q4-1) 2-4 코드에서 optim.SGD를 사용하지않고 코드를 짠다면?

for 구문에 weight를 업데이트 하는 (w.data = w.data - lr * w.grad.item( )) 과

다시 초기화하는 (w.grad.data.zero_( ))를 넣으면 된다. 

# Q4-2) Gradient descent 코드 구현 (adam, rmsprop, sgd에 momentum 추가) 

In [2]:
#Adam
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: 0.665, b: 0.123 Cost: 0.411208
Epoch  100/1000 W: 0.905, b: 0.212 Cost: 0.006594
Epoch  200/1000 W: 0.958, b: 0.092 Cost: 0.001258
Epoch  300/1000 W: 0.987, b: 0.030 Cost: 0.000131
Epoch  400/1000 W: 0.997, b: 0.007 Cost: 0.000008
Epoch  500/1000 W: 0.999, b: 0.001 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000


In [3]:
#RMS prop
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegressionModel()

optimizer = optim.RMSprop(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train)
    
    cost = F.mse_loss(prediction, y_train)
    
    optimizer.zero_grad() # 미분값이
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: -0.671, b: -0.275 Cost: 17.438334
Epoch  100/1000 W: 0.502, b: 0.836 Cost: 0.195549
Epoch  200/1000 W: 0.632, b: 0.810 Cost: 0.096204
Epoch  300/1000 W: 0.704, b: 0.657 Cost: 0.063028
Epoch  400/1000 W: 0.791, b: 0.464 Cost: 0.031587
Epoch  500/1000 W: 0.881, b: 0.264 Cost: 0.010243
Epoch  600/1000 W: 0.953, b: 0.105 Cost: 0.001624
Epoch  700/1000 W: 0.990, b: 0.023 Cost: 0.000078
Epoch  800/1000 W: 0.999, b: 0.002 Cost: 0.000001
Epoch  900/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch 1000/1000 W: 0.982, b: -0.018 Cost: 0.001760


In [4]:
#SGD에 Momentum 추가 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegressionModel()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) 

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: 0.683, b: 0.406 Cost: 0.141928
Epoch  100/1000 W: 0.991, b: 0.023 Cost: 0.000081
Epoch  200/1000 W: 1.000, b: 0.001 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  400/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: 0.000 Cost: 0.000000
